# Download OASIS DATASet

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ninadaithal/imagesoasis")

print("Path to dataset files:", path)

/home/user/miniforge3/envs/retfound/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/user/.cache/kagglehub/datasets/ninadaithal/imagesoasis/versions/1


In [2]:
import torch
import os
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from PIL import Image
import torch.nn as nn

### Load Dataset
Data_size: 224 x 224
1. Non demented: 6,7222
2. mild demented: 5002
3. moderate demented: 488
4. very demented: 1,3725

In [3]:
# import dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.CenterCrop(size=(224, 224)),  # resize to 224x224
    transforms.ToTensor(),          # convert PIL image to PyTorch tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )  # standard ImageNet normalization
])

train_dataset = datasets.ImageFolder(root='data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
finetune_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [4]:
# Model settings
from timm.optim import optim_factory
from Model.ViT.models_vit import VisionTransformer
from Model.ViT.models_mae import MaskedAutoencoderViT
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

weight_path = os.path.join("Model", "ViT", "mae_pretrain_vit_base.pth")
model_mae = MaskedAutoencoderViT(embed_dim=768, depth=12, num_heads=12)
check_point = torch.load(weight_path)
check_point_model = check_point['model']
model_mae.load_state_dict(check_point_model, strict=False)


param_groups = optim_factory.add_weight_decay(model_mae, 0.05)
optimizer = torch.optim.AdamW(param_groups, lr=1.5e-4, betas=(0.9, 0.95))

mseloss = nn.MSELoss()

model_mae = model_mae.to(device)

In [5]:
def load_checkpoints(epoch, model, optimizer, stage):
    checkpoint_path = f"checkpoints/RETfound/{stage}"
    
    if os.path.exists(checkpoint_path):
        print(f"Load checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']
        print(f"Loaded checkpoint from epoch {epoch}")

def save_checkpoints(epoch, model, optimizer, stage):
    checkpoint_path = f"checkpoints/ConvNeXtV2/{stage}"
    
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }, checkpoint_path + f"/checkpoint_{epoch}.pth")

# Pretrain fcmae

In [ ]:
#pretrain mae on imagenet
pretrain_epoch = 2000
start_epoch = 0


if start_epoch:
    load_checkpoints(start_epoch, model_mae, optimizer, stage="imagenet")

for epoch in range(start_epoch, pretrain_epoch):
    torch.cuda.empty_cache()
    epoch_loss = 0

    localtime = time.asctime( time.localtime(time.time()) )
    tqdm.write('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,pretrain_epoch,localtime))
    tqdm.write('-' * len('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,pretrain_epoch, localtime)))

    folder_name = os.path.join("see_image", "imagent")
    os.makedirs(folder_name, exist_ok=True)

    torch.cuda.empty_cache()
    
    for batch in tqdm(imagenet_loader):
        img, label = batch
        img = img.to(device)


            
        model_mae.train()
        model_autoencoder.eval()
        optimizer.zero_grad()


        loss, pred, mask = model_mae(img)
            
        image = pred[0].cpu().detach().numpy().transpose(1, 2, 0)
        img_np = img[0].cpu().detach().numpy().transpose(1, 2, 0)
        img_np = (img_np * 255).astype(np.uint8)
        pred_np = (image * 255).astype(np.uint8)

        cv2.imshow("pred img", pred_np)
        cv2.waitKey(1)
        cv2.imshow("targe img", img_np)
        cv2.waitKey(1)

        epoch_loss += loss
        loss.backward()
        optimizer.step()
        
    print(f'Epoch{epoch+1} loss : \n pretrain loss : {epoch_loss}')

    if (epoch+1) % 10 == 0:
        save_checkpoints(epoch, model_mae, optimizer, stage="imagenet")

In [ ]:
#pretrain mae on oasis dataset
pretrain_epoch = 2000
start_epoch = 0


if start_epoch:
    load_checkpoints(start_epoch-1, model_mae, optimizer, stage="pretrain")

for epoch in range(start_epoch, pretrain_epoch):
    torch.cuda.empty_cache()
    epoch_loss = 0

    localtime = time.asctime( time.localtime(time.time()) )
    tqdm.write('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,pretrain_epoch,localtime))
    tqdm.write('-' * len('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1,pretrain_epoch, localtime)))

    folder_name = os.path.join("see_image", "pre-train")
    os.makedirs(folder_name, exist_ok=True)

    torch.cuda.empty_cache()
    
    for batch in tqdm(train_loader):
        img, label = batch
        img = img.to(device)


        loss, pred, mask = model_mae(img)
            
        image = pred[0].cpu().detach().numpy().transpose(1, 2, 0)
        img_np = img[0].cpu().detach().numpy().transpose(1, 2, 0)
        img_np = (img_np * 255).astype(np.uint8)
        pred_np = (image * 255).astype(np.uint8)

        cv2.imshow("pred img", pred_np)
        cv2.waitKey(1)
        cv2.imshow("targe img", img_np)
        cv2.waitKey(1)
            
        epoch_loss += loss
        loss.backward()
        optimizer.step()
        
    print(f'Epoch{epoch+1} loss : \n pretrain loss : {epoch_loss}')

    if (epoch+1) % 10 == 0:
        save_checkpoints(epoch+1, model=model_mae, optimizer=optimizer, stage="pretrain")

In [6]:
from transformers import ViTForImageClassification

model_autoencoder = ViTForImageClassification.from_pretrained('facebook/deit-base-patch16-224')
model_autoencoder.classifier = nn.Linear(model_autoencoder.config.hidden_size, 4)
model_autoencoder = model_autoencoder.to(device)

param_groups = optim_factory.add_weight_decay(model_autoencoder, 0.05)
optimizer = torch.optim.AdamW(param_groups, lr=1.5e-4, betas=(0.9, 0.95))

In [7]:
# finetune classification
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
model_mae = None
torch.cuda.empty_cache()
pretrain_epoch = 25
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
start_epoch = 0

ce_loss = nn.CrossEntropyLoss()

if start_epoch:
    load_checkpoints(start_epoch-1, model_mae, optimizer, stage="finetune")

for epoch in range(start_epoch, pretrain_epoch):
    torch.cuda.empty_cache()
    epoch_loss = 0

    localtime = time.asctime( time.localtime(time.time()) )
    print('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1, pretrain_epoch,localtime))
    print('-' * len('Epoch: {}/{} --- < Starting Time : {} >'.format(epoch + 1, pretrain_epoch,localtime)))

    folder_name = os.path.join("see_image", "warm")
    os.makedirs(folder_name, exist_ok=True)

    torch.cuda.empty_cache()

    #model_mae.eval()
    model_autoencoder.train()
    
    
    for batch in tqdm(finetune_loader):
        img, label = batch
        img, label = img.to(device), label.to(device)


        outputs = model_autoencoder(img)
        loss = ce_loss(outputs.logits, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        
        
    print(f'Epoch{epoch+1} loss : \n pretrain loss : {epoch_loss}')
    epoch_loss = epoch_loss / len(finetune_loader)
    
    with open("Deit_train.txt", 'a+') as f:
        f.write(f"{epoch_loss}\n")
        
    #validation



    test_loss = 0
    model_autoencoder.eval()

    # Initialize lists to store true labels and predictions
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            img, label = batch
            img, label = img.to(device), label.to(device)

            #image_features = model.encode_image(img)
            #text_features = model.encode_text(clip.tokenize(cathegories).to(device))

            outputs = model_autoencoder(img)
            #probs = logits_img.softmax(dim=-1).cpu().numpy()
            loss = ce_loss(outputs.logits, label)

            
            # Store the true labels and predictions
            preds = outputs.logits.argmax(dim=1).cpu().numpy()
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(preds)
            test_loss += loss.item()
    test_loss = test_loss / len(val_loader)
    with open("Deit_val.txt",  "a+") as f:
        f.write(f"{test_loss}\n")
            
    # Calculate precision, recall, F1 score, and accuracy
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    print("Test loss:", test_loss)

Epoch: 1/25 --- < Starting Time : Sat Dec 21 17:01:58 2024 >
------------------------------------------------------------


100%|██████████| 8644/8644 [09:34<00:00, 15.04it/s]


Epoch1 loss : 
 pretrain loss : 2341.689697265625


100%|██████████| 2161/2161 [01:04<00:00, 33.27it/s]


Precision: 0.9436
Recall: 0.9452
F1 Score: 0.9425
Accuracy: 0.9452
Test loss: 0.17036653988232275
Epoch: 2/25 --- < Starting Time : Sat Dec 21 17:12:40 2024 >
------------------------------------------------------------


...

Precision: 0.9872
Recall: 0.9868
F1 Score: 0.9869
Accuracy: 0.9868
Test loss: 0.04626175354122405
Epoch: 25/25 --- < Starting Time : Sat Dec 21 21:22:44 2024 >
-------------------------------------------------------------


100%|██████████| 8644/8644 [09:45<00:00, 14.77it/s]


Epoch25 loss : 
 pretrain loss : 340.0849609375


100%|██████████| 2161/2161 [01:03<00:00, 34.19it/s]

Precision: 0.9929
Recall: 0.9928
F1 Score: 0.9928
Accuracy: 0.9928
Test loss: 0.022847359690412378
